In [25]:
import torch
import torchvision.models as models
import torch.jit
import torch.utils.benchmark as benchmark

# Load ResNet-152 (you can use a pretrained or random initialized model)
model = models.resnet152(pretrained=True)

# Set the model to evaluation mode
model.eval()

# Create a random input tensor to use for tracing
input_tensor = torch.randn(1, 3, 224, 224)  # Assuming 3-channel images of size 224x224

# Script the model using torch.jit.script
scripted_model = torch.jit.script(model)

# Generate random input data
input_data = torch.randn(32, 3, 224, 224)  # Creating a batch of 32 input samples for benchmarking

# Define a benchmark timer for the unscripted model
timer_unscripted = benchmark.Timer(
    stmt="model(input_data)",
    setup="from __main__ import model, input_data",
    num_threads=1,  # You can adjust the number of threads for multi-threaded inference
)

# Define a benchmark timer for the scripted modelf
timer_scripted = benchmark.Timer(
    stmt="scripted_model(input_data)",
    setup="from __main__ import scripted_model, input_data",
    num_threads=1,  # You can adjust the number of threads for multi-threaded inference
)

# Run the benchmarks
unscripted_time = timer_unscripted.timeit(10)  # Run 10 iterations
scripted_time = timer_scripted.timeit(10)  # Run 10 iterations

# Extract the timing values from Measurement objects
unscripted_time_mean = unscripted_time.mean
scripted_time_mean = scripted_time.mean

# Calculate the percentage increase in efficiency
efficiency_increase = (unscripted_time_mean / scripted_time_mean - 1) * 100

# Print the benchmark results
print(f"Unscripted Model Time: {unscripted_time_mean} seconds")
print(f"Scripted Model Time: {scripted_time_mean} seconds")
print(f"Percentage Increase in Efficiency: {efficiency_increase:.2f}%")


Unscripted Model Time: 7.203471150000405 seconds
Scripted Model Time: 7.004129645799549 seconds
Percentage Increase in Efficiency: 2.85%
